#  Library 호출

# 주석 해제 후 파일 경로 설정 필수

In [197]:
pip install sklearn

In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [200]:
pip install matplotlib

In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.mixture import GaussianMixture
import numpy as np

In [212]:
### Input data 경로 설정#####
#### 최종 데이터 경로 설정####
data_set=pd.read_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\데이터원본\\최종데이터.xlsx')

In [204]:
data_set=data_set.sort_values(by='가중치',ascending=False)

In [205]:
data_high20=data_set.copy()

In [206]:

data_high20=data_set.copy()
high_20=int(len(data_high20.iloc[:,0])*0.3) ## 상위 30% 뽑음 

print(high_20)

data_high20=data_high20.iloc[0:high_20,]
data_result_able=data_high20.copy() ## 비행 불가능 지역 제거 안함

980


In [168]:
data_result_en=data_set.copy() ## 비행 불가능 지역 제거
data_result_en=data_result_en[data_result_en['비행불가능여부']==0]

data_high20_en=data_result_en.copy()
high_20=int(len(data_high20_en.iloc[:,0])*0.3) ## 상위 30% 뽑음 

print(high_20)

data_high20_en=data_high20_en.iloc[0:high_20,]
data_result_en=data_high20_en.copy()

731


# Silhouette 평가

In [176]:
def eval_s(data_set):
    data=data_set.loc[:,['LAT','LON']]
    average_score=silhouette_score(data,data_set['cluster']) ## 입력된 데이터 셋을 cluster label 부분과 데이터 부분으로 나누어 성능평가
    return average_score

# KMEANS

공역제외안함

In [172]:
data_result_able_kmeans=data_result_able.copy()
data_result_able_kmeans=data_result_able_kmeans.loc[:,['LAT','LON']]

kmeans=KMeans(n_clusters=52,init='k-means++',max_iter=300,random_state=0)
kmeans.fit(data_result_able_kmeans)

data_result_able_kmeans['cluster']=kmeans.labels_
score=eval_s(data_result_able_kmeans)


print('score=',score)
data_result_able_kmeans_plt=data_result_able_kmeans.iloc[:,0:2]
score_samples=silhouette_samples(data_result_able_kmeans_plt,data_result_able_kmeans['cluster'])

data_result_able_kmeans['silhouette-coeff']=score_samples
## 클러스터별 실루엣 계수 표준편차, 분산 확인하기 위해 태블로로 시각화 -> 파일로 저장후 태블로에서 염
g=data_result_able_kmeans[['cluster','silhouette-coeff']].groupby(['cluster']).mean().sort_values(by='silhouette-coeff')


#################################백업파일 만들때 사용########################################
#g.to_excel('분산표준편차 태블로 시각화용 경로설정')
#data_result_able_kmeans.to_excel('군집화결과경로설정')

score= 0.43711409139354857


# KMeans

공역제외함

In [154]:
data_result_en_kmeans=data_result_en.copy()
data_result_en_kmeans=data_result_en_kmeans.loc[:,['LAT','LON']]
kmeans=KMeans(n_clusters=52,init='k-means++',max_iter=300,random_state=0)
kmeans.fit(data_result_en_kmeans)


data_result_en_kmeans['cluster']=kmeans.labels_

score=eval_s(data_result_en_kmeans)

print('score=',score)

data_result_en_kmeans_plt=data_result_en_kmeans.iloc[:,0:2]

score_samples=silhouette_samples(data_result_en_kmeans_plt,data_result_en_kmeans['cluster'])
data_result_en_kmeans['silhouette-coeff']=score_samples
## 클러스터별 실루엣 계수 표준편차, 분산 확인하기 위해 태블로로 시각화 -> 파일로 저장후 태블로에서 염
g=data_result_en_kmeans[['cluster','silhouette-coeff']].groupby(['cluster']).mean().sort_values(by='silhouette-coeff')


##################################백업팡리 만들때 사용####################################
#g.to_excel('태블로용 분산, 표준편차 데이터 백업 경로')
#data_result_en_kmeans.to_excel('클러스터링 결과 백업 경로')

score= 0.44816697637029473


In [155]:
print(np.var(g),np.std(g)) ## 표준편차 , 분산 확인

silhouette-coeff    0.063935
dtype: float64 silhouette-coeff    0.252853
dtype: float64


# DBSCAN

공역제외안함

In [174]:
list_array=np.arange(1,0,-0.001)
## eps 파라미터 용 배열 생성

In [175]:
# 파라미터 튜닝 용 함수 , 데이터와 eps,min_samples 인자를 받고 클러스터 개수를 return함
def db_scan(dbscan_data,ep,samples):
    dbscan=DBSCAN(eps=ep,min_samples=samples,metric='euclidean')
    dbscan_labels=dbscan.fit_predict(dbscan_data)
    len_cluster=np.unique(dbscan_labels)
    num_cluster=np.max(len_cluster)
    return num_cluster

In [ ]:
parameter_tuning_min_samples=[]
parameter_tuning_eps=[]
data_result_able_dbscan=data_result_able.copy()
data_result_able_dbscan=data_result_able_dbscan.loc[:,['LAT','LON']]
score_list=[]
########################parameter tuning##################
for samples in range(1,8,1):
    for ep in list_array:
        len_cluster=db_scan(data_result_able_dbscan,ep,samples)
        if len_cluster!=51:
            pass
        else :
            parameter_tuning_min_samples.append(samples)
            parameter_tuning_eps.append(ep)
            
################################################################
#######################튜닝된 파라미터별 Score 확인###################################
for w, q in zip(parameter_tuning_min_samples,parameter_tuning_eps):
    dbscan=DBSCAN(eps=q,min_samples=w,metric='euclidean')
    dbscan_labels=dbscan.fit_predict(data_result_able_dbscan)
    dbscan_data=data_result_able_dbscan.copy()
    dbscan_data['cluster']=dbscan_labels
    dbscan_data=dbscan_data[dbscan_data['cluster']!=-1] #outlier 제거 
    score=eval_s(dbscan_data)
    score_list.append(score)    
    
max_score=score_list.index(np.max(score)) #가장 높은 score 파라미터 선택
print(max_score)
a=parameter_tuning_eps[max_score]
b=parameter_tuning_min_samples[max_score]

###########################################################################################

dbscan=DBSCAN(eps=a,min_samples=b,metric='euclidean')
dbscan_labels=dbscan.fit_predict(data_result_able_dbscan)
dbscan_data=data_result_able_dbscan.copy()
dbscan_data['cluster']=dbscan_labels
dbscan_data=dbscan_data[dbscan_data['cluster']!=-1] # outlier 제거

############클러스터링 결과 데이터 백업 파일 경로 설정###############################
#dbscan_data.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210827\\dbscan\\dbscan_able.xlsx')
#################################################################
##########################evaluation##################################

#dbscan_data.drop(['dbscan_labels'],axis=1,inplace=True)
score=eval_s(dbscan_data)
print('score=',score)
dbscan_data_plt=dbscan_data.iloc[:,0:2]
score_samples=silhouette_samples(dbscan_data_plt,dbscan_data['cluster'])
dbscan_data['silhouette-coeff']=score_samples

## 클러스터별 실루엣 계수 표준편차, 분산 확인하기 위해 태블로로 시각화 -> 파일로 저장후 태블로에서 염
g=dbscan_data[['cluster','silhouette-coeff']].groupby(['cluster']).mean().sort_values(by='silhouette-coeff')



#######태블로용 데이터 백업 경로 설정 #########
#g.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\평균분산표준편차\\kmeans.xlsx')


# DBSCAN

공역제거함

In [156]:
parameter_tuning_min_samples=[]
parameter_tuning_eps=[]
data_result_en_dbscan=data_result_en.copy()
data_result_en_dbscan=data_result_en_dbscan.loc[:,['LAT','LON']]
score_list=[]
########################parameter tuning##################
for j in range(1,5,1):
    print(j,'^'*100)
    for i in list_array:
        len_cluster=db_scan(data_result_en_dbscan,i,j)
        print(len_cluster)
        if len_cluster==51:
            print('51개 minsamples',j)
            parameter_tuning_min_samples.append(j)
            parameter_tuning_eps.append(i)
        else:
            pass
################################################################
#######################튜닝된 파라미터 별 score 확인##################################

for a, b in zip(parameter_tuning_min_samples,parameter_tuning_eps):
    dbscan=DBSCAN(eps=b,min_samples=a,metric='euclidean')
    dbscan_labels=dbscan.fit_predict(data_result_en_dbscan)
    dbscan_data=data_result_en_dbscan.copy()
    dbscan_data['cluster']=dbscan_labels
    dbscan_data=dbscan_data[dbscan_data['cluster']!=-1]
    score=eval_s(dbscan_data)
    score_list.append(score)
    
max_score=score_list.index(np.max(score_list))#가장 높은 score 파라미터 선택
a=parameter_tuning_eps[max_score]
b=parameter_tuning_min_samples[max_score]
###################################################################################



dbscan=DBSCAN(eps=a,min_samples=b,metric='euclidean')
dbscan_labels=dbscan.fit_predict(data_result_en_dbscan)
dbscan_data=data_result_en_dbscan.copy()
dbscan_data['cluster']=dbscan_labels
dbscan_data=dbscan_data[dbscan_data['cluster']!=-1]

############클러스터링 결과 데이터 백업 파일 경로 설정###############################
#dbscan_data.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\dbscan\\dbscan_en.xlsx')

#################################################################
##########################evaluation##################################
try:
    dbscan_data.drop(['dbscan_labels'],axis=1,inplace=True)
except:
    pass

score=eval_s(dbscan_data)
print('score=',score)

dbscan_data_plt=dbscan_data.iloc[:,0:2]

score_samples=silhouette_samples(dbscan_data_plt,dbscan_data['cluster'])
dbscan_data['silhouette-coeff']=score_samples

## 클러스터별 실루엣 계수 표준편차, 분산 확인하기 위해 태블로로 시각화 -> 파일로 저장후 태블로에서 염
g=dbscan_data[['cluster','silhouette-coeff']].groupby(['cluster']).mean().sort_values(by='silhouette-coeff')


#######태블로용 데이터 백업 경로 설정 #########
##g.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\평균분산표준편차\\dbscan.xlsx')

            

1 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4

5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
11
11
11
11
13
14
13
12
13
14
13
13
14
14
15
14
16
18
17
19
19
17
22
22
24
26
26
29
29
31
35
32
34
37
31
30
21
7
3
1
0
0
-1
-1
-1
-1
score= 0.1670270737561398


In [148]:
print(np.var(g),np.std(g))

silhouette-coeff    0.135507
dtype: float64 silhouette-coeff    0.368113
dtype: float64 silhouette-coeff    0.280529
dtype: float64


# GMM

##### 공역제거안함

In [177]:
###############################GMM 군집화################################################
data_result_able_gmm=data_result_able.copy()
data_result_able_gmm=data_result_able_gmm.loc[:,['LAT','LON']]
gmm = GaussianMixture(n_components=52,random_state=0).fit(data_result_able_gmm)
gmm_cluster_label=gmm.predict(data_result_able_gmm)
data_result_able_gmm['cluster']=gmm_cluster_label
##################################################################################

############클러스터링 결과 데이터 백업 파일 경로 설정###############################
#data_result_able_gmm.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210827\\gmm\\gmm_able.xlsx')
eval_gmm=eval_s(data_result_able_gmm)
print('score=',eval_gmm)
data_for_plt=data_result_able_gmm.iloc[:,0:2]

score_samples=silhouette_samples(data_for_plt,data_result_able_gmm['cluster'])
data_result_able_gmm['silhouette-coeff']=score_samples
## 클러스터별 실루엣 계수 표준편차, 분산 확인하기 위해 태블로로 시각화 -> 파일로 저장후 태블로에서 염
g=data_result_able_gmm[['cluster','silhouette-coeff']].groupby(['cluster']).mean().sort_values(by='silhouette-coeff')




#######태블로용 데이터 백업 경로 설정 #########
##g.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\평균분산표준편차\\GMM.xlsx')

score= 0.3761551051157046


# GMM

##### 공역제거함

In [178]:
###############################GMM 군집화################################################
data_result_en_gmm=data_result_en.copy()
data_result_en_gmm=data_result_en_gmm.loc[:,['LAT','LON']]
gmm = GaussianMixture(n_components=52,random_state=0).fit(data_result_en_gmm)
gmm_cluster_label=gmm.predict(data_result_en_gmm)
data_result_en_gmm['cluster']=gmm_cluster_label
##################################################################################



############클러스터링 결과 데이터 백업 파일 경로 설정###############################
#data_result_en_gmm.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\gmm\\gmm_en.xlsx')



eval_gmm=eval_s(data_result_en_gmm)
print('score=',eval_gmm)
data_for_plt=data_result_en_gmm.iloc[:,0:2]

score_samples=silhouette_samples(data_for_plt,data_result_en_gmm['cluster'])
data_result_en_gmm['silhouette-coeff']=score_samples

## 클러스터별 실루엣 계수 표준편차, 분산 확인하기 위해 태블로로 시각화 -> 파일로 저장후 태블로에서 염
g=data_result_en_gmm[['cluster','silhouette-coeff']].groupby(['cluster']).mean().sort_values(by='silhouette-coeff')
#######태블로용 데이터 백업 경로 설정 #########
#g.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\평균분산표준편차\\GMM.xlsx')

score= 0.40410599831989896


In [152]:
print(np.var(g),np.std(g))

silhouette-coeff    0.071564
dtype: float64 silhouette-coeff    0.267515
dtype: float64


# 데이터 전처리 -> 엑셀 함수 이용

In [157]:
## 2번째 input data 경로 설정 ##
#### kmeans_en_결과도출용 데이터 경로 설정
#data_result=pd.read_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\최종결과\\kmeans_en_결과도출용.xlsx')

In [158]:
data_result

,Unnamed: 0,LAT,LON,cluster,silhouette-coeff,총합,주소
0,3099,35.352822,129.040607,44,0.558845,24033.869407,경상남도 양산시 신기동
1,1362,37.657852,127.300719,43,0.422916,22813.655793,경기도 남양주시 화도읍
2,1540,37.389034,127.229802,17,0.650665,22122.955809,경기도 광주시 오포읍
3,1361,37.725885,127.189674,43,0.418650,20160.338163,경기도 남양주시 진접읍
4,3055,35.222075,128.926749,47,0.160801,17956.457536,경상남도 김해시 지내동
...,...,...,...,...,...,...,...
726,584,35.143789,129.114737,14,0.447586,6729.248328,부산광역시 수영구 남천동
727,184,37.571124,126.959077,36,0.439581,6718.849400,서울특별시 서대문구 냉천동
728,155,37.624821,127.073846,7,0.630104,6717.319384,서울특별시 노원구 공릉동
729,1467,37.771416,126.778354,50,0.414256,3550.601073,경기도 파주시 파주읍


In [159]:
d_result=pd.DataFrame([])
for i in range (53):
    data_find_king=data_result[data_result['cluster']==i].sort_values(by='총합',ascending=False)
    try:
        data_find_king_label=data_find_king.iloc[0,:] ## cluster 별로  오름 차순 정렬 후 가장 큰 값 추출
    except:
        pass
    d_result=pd.concat([d_result,data_find_king_label],axis=1)
    print(data_find_king)

     Unnamed: 0        LAT         LON  cluster  silhouette-coeff  \
94         1323  37.626388  126.838598        0          0.591237   
145        1317  37.622623  126.818077        0          0.428013   
154        1307  37.659413  126.829614        0          0.464286   
209        1309  37.644926  126.854348        0          0.611279   
227        1316  37.687988  126.862554        0          0.442781   
238        1320  37.618448  126.825322        0          0.494154   
275        1318  37.644650  126.833541        0          0.569640   
361        1308  37.677407  126.875845        0          0.503009   
386        1322  37.615136  126.833004        0          0.484917   
403        1315  37.703360  126.903748        0          0.311407   
413        1326  37.677340  126.813707        0         -0.029389   
418        1319  37.630569  126.835720        0          0.590848   
691        1310  37.657119  126.836298        0          0.540009   

               총합                

191  1979.800851  강원도 춘천시 신동면  
     Unnamed: 0        LAT         LON  cluster  silhouette-coeff  \
108        2964  35.251926  128.667572       16          0.688304   
111        2965  35.232683  128.650474       16          0.716854   
126        2968  35.215865  128.708767       16          0.518780   
132        2967  35.216240  128.687193       16          0.637776   
338        2961  35.262031  128.622863       16          0.700173   
341        2962  35.255424  128.612830       16          0.693238   
392        2975  35.154344  128.686897       16          0.430545   
555        2970  35.189691  128.695062       16          0.540471   
556        2959  35.347780  128.607041       16          0.521366   

               총합                 주소  
108  10948.312364    경상남도 창원시 의창구 용동  
111  10888.411577   경상남도 창원시 성산구 반송동  
126  10689.695418   경상남도 창원시 성산구 토월동  
132  10586.368377  경상남도 창원시 성산구 가음정동  
338   8693.654489    경상남도 창원시 의창구 북동  
341   8673.381787   경상남도 창원시 의창구 덕정동  
392 

     Unnamed: 0        LAT         LON  cluster  silhouette-coeff  \
67         1819  36.635515  127.432547       35          0.730080   
118        1821  36.628862  127.433046       35          0.752257   
259        1804  36.608867  127.501376       35          0.727326   
530        1813  36.608705  127.489178       35          0.770640   
615        1823  36.626660  127.426008       35          0.728918   
629        1812  36.617328  127.472035       35          0.795899   
610        1803  36.631671  127.493227       35          0.718055   

               총합                  주소  
67   11564.428792    충청북도 청주시 흥덕구 복대동  
118  10760.411033    충청북도 청주시 흥덕구 가경동  
259   9263.868764    충청북도 청주시 상당구 용암동  
530   7618.431342    충청북도 청주시 서원구 미평동  
615   7203.274183    충청북도 청주시 흥덕구 강서동  
629   7146.417474    충청북도 청주시 서원구 산남동  
610   2856.395782  충청북도 청주시 상당구 북문로1가  
     Unnamed: 0        LAT         LON  cluster  silhouette-coeff  \
49          177  37.592242  126.915735       36          0

In [160]:
d_result=d_result.T
d_result.drop_duplicates(inplace=True)
# 혹시모를 중복값 제거 

In [162]:
##최종 52개 버티포트 위치 백업파일경로설정#

#d_result.to_excel('C:\\Users\\qjawl\\Desktop\\빅데이터\\빅데이터프로젝트\\UAM\\20210829\\최종결과\\centriod.xlsx')